<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/Fine_Tune_AI_Text_Generation_GPT_Neo_Model_with_Custom_Dataset_%26_Upload_to_Hugging_Face_Hub_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget -O prompts.csv https://raw.githubusercontent.com/krea-ai/open-prompts/main/data/1k.csv

--2022-10-31 17:40:48--  https://raw.githubusercontent.com/krea-ai/open-prompts/main/data/1k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1027887 (1004K) [text/plain]
Saving to: ‘prompts.csv’

prompts.csv         100%[===================>]   1004K  --.-KB/s    in 0.005s  

2022-10-31 17:40:51 (186 MB/s) - ‘prompts.csv’ saved [1027887/1027887]



In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
!nvidia-smi

Mon Oct 31 16:09:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Stable Diffusion Prompt

In [3]:
prompts = pd.read_csv("prompts.csv")

In [4]:
prompts

,prompt,raw_data
0,A portrait photo of a kangaroo wearing an oran...,"{""image_uri"": ""PENDING"", ""modifiers"": [""portra..."
1,inmates with cow heads inside a jailcell,"{""image_uri"": ""PENDING"", ""modifiers"": [""inmate..."
2,"daguerrotype of a corgi astronaut on the moon,...","{""image_uri"": ""PENDING"", ""modifiers"": [""daguer..."
3,totem animal tribal chaman vodoo mask feather ...,"{""image_uri"": ""PENDING"", ""modifiers"": [""totem ..."
4,p. cubensis,"{""image_uri"": ""PENDING"", ""modifiers"": [""p cube..."
...,...,...
993,"warrior tattoo, magnificent","{""image_uri"": ""PENDING"", ""modifiers"": [""warrio..."
994,"nematode worms find a plant cell, microscopy, ...","{""image_uri"": ""PENDING"", ""modifiers"": [""nemato..."
995,hyperrealism close-up portrait of beautiful me...,"{""image_uri"": ""PENDING"", ""modifiers"": [""hyperr..."
996,an extremely detailed masterpiece of a mouse w...,"{""image_uri"": ""PENDING"", ""modifiers"": [""extrem..."


## Exclude Tokens having less than 6 words

In [5]:
prompts_gt6 = prompts.loc[prompts.prompt.str.split(' ').str.len() > 6]

A robust Python tool for text-based AI training and generation using OpenAI's GPT-2 and EleutherAI's GPT Neo/GPT-3 architecture.

aitextgen is a Python package that leverages PyTorch, Hugging Face Transformers and pytorch-lightning with specific optimizations for text generation using GPT-2, plus many added features. It is the successor to textgenrnn and gpt-2-simple, taking the best of both packages:


https://docs.aitextgen.io/

In [6]:
! pip install aitextgen -q

     |████████████████████████████████| 572 kB 30.7 MB/s 
     |████████████████████████████████| 5.3 MB 6.5 MB/s 
     |████████████████████████████████| 87 kB 7.3 MB/s 
     |████████████████████████████████| 708 kB 54.6 MB/s 
     |████████████████████████████████| 529 kB 74.1 MB/s 
     |████████████████████████████████| 7.6 MB 62.3 MB/s 
     |████████████████████████████████| 163 kB 69.7 MB/s 


In [7]:
model="EleutherAI/gpt-neo-125M"

In [8]:
prompts_gt6 = prompts_gt6.drop('raw_data', axis = 1).reset_index(drop=True)

In [9]:
prompts_gt6.to_csv("input_text_cleaned.txt", columns=["prompt"], header=False, index=False)

In [10]:
from aitextgen.TokenDataset import TokenDataset

In [11]:
data = TokenDataset('input_text_cleaned.txt', line_by_line=True)

  0%|          | 0/896 [00:00<?, ?it/s]

INFO:aitextgen.TokenDataset:Encoding 896 sets of tokens from input_text_cleaned.txt.


In [12]:
data

TokenDataset containing 40,890 subsets loaded from line-by-line file at input_text_cleaned.txt.

In [13]:
from aitextgen import aitextgen

In [14]:
ai = aitextgen(model = model,  to_gpu=True)

INFO:aitextgen:Downloading EleutherAI/gpt-neo-125M model to /aitextgen.


Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

INFO:aitextgen:Using the tokenizer for EleutherAI/gpt-neo-125M.


Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

loading file vocab.json from cache at aitextgen/models--EleutherAI--gpt-neo-125M/snapshots/324e21bd3b56dfecba4308ab6ec147b588df23af/vocab.json
loading file merges.txt from cache at aitextgen/models--EleutherAI--gpt-neo-125M/snapshots/324e21bd3b56dfecba4308ab6ec147b588df23af/merges.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at aitextgen/models--EleutherAI--gpt-neo-125M/snapshots/324e21bd3b56dfecba4308ab6ec147b588df23af/special_tokens_map.json
loading file tokenizer_config.json from cache at aitextgen/models--EleutherAI--gpt-neo-125M/snapshots/324e21bd3b56dfecba4308ab6ec147b588df23af/tokenizer_config.json
INFO:aitextgen:GPTNeo loaded with 125M parameters.


In [ ]:
ai.train('input_text_cleaned.txt',
         line_by_line=True,
         from_cache=False,
         num_steps=500,
         generate_every=100,
         save_every=500,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1, 
         )

INFO:aitextgen:Loading text from input_text_cleaned.txt with generation length of 2048.


  0%|          | 0/896 [00:00<?, ?it/s]

INFO:aitextgen.TokenDataset:Encoding 896 sets of tokens from input_text_cleaned.txt.
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=-1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=-1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:259: LightningDeprecationWarning: The `Callback.on_batch_end` hook was deprecated in v1.6 and will be removed in v1.8. Please use `Callback.on_tr

  0%|          | 0/500 [00:00<?, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


100 steps reached: generating sample texts.
 artstation



A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


200 steps reached: generating sample texts.
, beautiful, award winning photography, award winning, HD, 4k, 8k"



In [ ]:
ai.save()

In [ ]:
prompt_ai = aitextgen(model_folder = '.', to_gpu=True)